# Shark Attack Project #

## Goal

This project has 3 main questions to answers as a goal:

1. What are the top 10 places in the world with the highest number of shark attacks historically?

1. What are the top 10 places in the world with the highest number of death by shark attack historycally? Is it the same answer as question 1?

1. Is there a difference in death rate (number of deaths/number of attacks) between places?

## Coding

### Importing modules that will be used throughout the code

In [63]:
import pandas as pd
import numpy as np
import re

In [65]:
# Importing raw data base and assigning the DataFrame to a variable called raw_data
raw_data = pd.read_csv('data/attacks.csv')
# Checking which columns have non null values
raw_data.info()
# raw_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

In [66]:
# Removing columns not useful from for the analysis and applying to a new variable called raw_data_1. 
columns_to_remove = [
                    'Unnamed: 22',
                    'Unnamed: 23'
                    ]
raw_data_1 = raw_data.drop(columns_to_remove,axis=1)
# df.columns = [x.lower() for x in df.columns]
new_columns_dict = {columns:columns.strip().replace(' ', '_').title() for columns in raw_data_1.columns}
raw_data_1.rename(columns = new_columns_dict, inplace = True)

In [67]:
# Droping values that has NaN fullfiled in 'important' columns
raw_data_1.dropna(subset = ['Case_Number','Date'],inplace = True)

In [68]:
## Checking the values in column Year
raw_data_1.Year.describe()

count    6299.000000
mean     1927.275917
std       281.138485
min         0.000000
25%      1942.000000
50%      1977.000000
75%      2005.000000
max      2018.000000
Name: Year, dtype: float64

In [71]:
# Checking column date where Year = 0, in an attempt to treat the cases with Year = 0
raw_data_1[raw_data_1.Year == 0].Date.unique()

array(['Ca. 214 B.C.', 'Ca. 336.B.C..', '493 B.C.', 'Ca. 725 B.C.',
       'Before 1939', '1990 or 1991', 'Before 2016', 'Before Oct-2009',
       'Before 1934', '2009?', 'Before 1930', '1880-1899', 'Before 1909',
       'Before 2012', 'Before 1916', 'Between   1951-1963', 'Before 1908',
       'Before 1900', 'Before 1876', 'Before 2011', 'Before 2009',
       'Beforer 1994', 'Before 1963', '1896-1913', 'Before 1936',
       'Before 08-Jun-1912', 'Before 1911', 'Before 1901',
       'No date, late 1960s', 'Before 2006', 'Before 2003', 'Before 2004',
       'Before 1962', '1950s', 'No date, Before 1963', '2003?', 'No date',
       'Before Feb-1998', 'No date, Before May-1996',
       'No date, Before Mar-1995', 'Before 1996',
       'No date, Before Aug-1989', 'No date, Before Aug-1987',
       'No date, Before 1987', 'No date, Before  1975',
       'No date, Before 1975', 'No date, Before 1969',
       'No date, Before 3-Jan-1967', 'No date, Before 8-May-1965',
       'No date, Before 

In [72]:
# Defining function to treat Year column using values from Date column
def treat_col_year(x):
    if 'B.C.' in x:
        x1 = re.sub('\D','', x)
        return -int(x1)
    else:
        try:
            x1 = re.sub('\D',"", x)
            return int(x1[len(x1)-4:])
        except:
            return np.nan

In [73]:
# Creating and Assigning to Adj_Year the value Year, then, treating the cases where Year = 0 with the defined fucntion above 
raw_data_1['Adj_Year'] = raw_data_1.Year
raw_data_1.loc[raw_data_1.Adj_Year == 0,'Adj_Year'] = raw_data_1.Date.map(treat_col_year)
raw_data_1.shape

(6301, 23)

In [74]:
# Dropping NaN values in Adj_Year column.
raw_data_1.dropna(subset = ['Adj_Year'],inplace = True)
raw_data_1.shape

(6290, 23)

In [75]:
# Checking unique registers in Sex column.
raw_data_1.Sex.unique()

array(['F', 'M', nan, 'M ', 'lli', 'N', '.'], dtype=object)

In [76]:
# Defining function to treat sex column
def treat_col_sex(x):
    if type(x) == str:
        x = x.strip().upper()
        list_sex = ['M','F']
        if x == 'N':
            x = 'M'
        else:
            pass
        if x in list_sex:
            return x
        else:
            return 'Not available'
    else:
        return 'Not available'

In [77]:
# Creating a new columns Adj_Gender to assign the tretated values from column Sex 
raw_data_1['Adj_Gender'] = ''
raw_data_1.loc[:,'Adj_Gender'] = raw_data_1.Sex.map(treat_col_sex)

In [78]:
# Checking unique values from Fatal_Y/N.
raw_data_1['Fatal_(Y/N)'].unique()

array(['N', 'Y', nan, 'M', 'UNKNOWN', '2017', ' N', 'N ', 'y'],
      dtype=object)

In [ ]:
# Defining function to treat Fatal (Y/N) column
def treat_col_fatal(x):
    if type(x) == str:
        x = x.strip().upper()
        list_YN = ['Y','N']
        if x == 'M':
            x = 'N'
        else:
            pass
        if x in list_YN:
            return x
        else:
            return 'Not available'
    else:
        return 'Not available'

In [79]:
# Creating a new columns Adj_Death to assign the tretated values from column Fatal_Y/N 
raw_data_1['Adj_Death'] = ''
raw_data_1.loc[:,'Adj_Death'] = raw_data_1['Fatal_(Y/N)'].map(treat_col_fatal)

In [80]:
# Checking Age unique values
raw_data_1.Age.unique()

array(['57', '11', '48', nan, '18', '52', '15', '12', '32', '10', '21',
       '34', '30', '60', '33', '29', '54', '41', '37', '56', '19', '25',
       '69', '38', '55', '35', '46', '45', '14', '40s', '28', '20', '24',
       '26', '49', '22', '7', '31', '17', '40', '13', '42', '3', '8',
       '50', '16', '82', '73', '20s', '68', '51', '39', '58', 'Teen',
       '47', '61', '65', '36', '66', '43', '60s', '9', '72', '59', '6',
       '27', '64', '23', '71', '44', '62', '63', '70', '18 months', '53',
       '30s', '50s', 'teen', '77', '74', '28 & 26', '5', '86', '18 or 20',
       '12 or 13', '46 & 34', '28, 23 & 30', 'Teens', '36 & 26',
       '8 or 10', '84', '� ', ' ', '30 or 36', '6�', '21 & ?', '75',
       '33 or 37', 'mid-30s', '23 & 20', ' 30', '7      &    31', ' 28',
       '20?', "60's", '32 & 30', '16 to 18', '87', '67', 'Elderly',
       'mid-20s', 'Ca. 33', '74 ', '45 ', '21 or 26', '20 ', '>50',
       '18 to 22', 'adult', '9 & 12', '? & 19', '9 months', '25 to 35',
     

In [81]:
# Defining function to treat Age column
def treat_col_age(x):
    if type(x) == str:
        try: 
            if '[Mm]onth[s ]'in x:
                x1 = re.sub('\D',"", x)
                x2 = x1/12
                return float (x2)
            else:
                x1 = re.sub('\D',"", x)
                return float (x)
        except:
            return np.nan
    else:
        return x

In [82]:
raw_data_1['Adj_Age'] = 0
raw_data_1.loc[:,'Adj_Age'] = raw_data_1['Age'].map(treat_col_age)
raw_data_1.Adj_Age.describe()

count    3368.000000
mean       27.366093
std        13.909223
min         1.000000
25%        17.000000
50%        24.000000
75%        35.000000
max        87.000000
Name: Adj_Age, dtype: float64

In [86]:
# Creating two columns with values 1/0 to count number of deaths vs. attacks
raw_data_1['Death_1_0'] = 0
raw_data_1['Attack_1_0'] = 1

In [87]:
# Assigning 1 to the filtered df, where Adj_death = 1
raw_data_1.loc[raw_data_1.Adj_Death == 'Y', 'Death_1_0'] = 1

In [88]:
# Creating Death_Rate column
raw_data_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6290 entries, 0 to 6301
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   Case_Number             6290 non-null   object  
 1   Date                    6290 non-null   object  
 2   Year                    6290 non-null   float64 
 3   Type                    6286 non-null   object  
 4   Country                 6240 non-null   object  
 5   Area                    5836 non-null   object  
 6   Location                5752 non-null   object  
 7   Activity                5746 non-null   object  
 8   Name                    6081 non-null   object  
 9   Sex                     5725 non-null   object  
 10  Age                     3470 non-null   object  
 11  Injury                  6262 non-null   object  
 12  Fatal_(Y/N)             5752 non-null   object  
 13  Time                    2948 non-null   object  
 14  Species                 

In [89]:
# Grouping years into new column called Group_Year
raw_data_1['Group_Year'] = np.nan
raw_data_1.loc[:,'Group_Year'] = pd.cut(raw_data_1.Adj_Year, bins = [-1000,0,1900,1910,1920,1930,1940,1950,1960,1970,\
                                                           1980,1990, 2000, 2010,2020])
# Grouping Age into new column called Group_Age
raw_data_1['Group_Age'] = np.nan
raw_data_1.loc[:,'Group_Age'] = pd.cut(raw_data_1.Adj_Age, bins = [12,18,60,90])
raw_data_1.Group_Year.unique()

[(2010, 2020], (2000, 2010], (1990, 2000], (1980, 1990], (1960, 1970], ..., (1930, 1940], (1920, 1930], (1910, 1920], (1900, 1910], (-1000, 0]]
Length: 14
Categories (14, interval[int64]): [(-1000, 0] < (0, 1900] < (1900, 1910] < (1910, 1920] ... (1980, 1990] < (1990, 2000] < (2000, 2010] < (2010, 2020]]

In [90]:
# Creating a summarized table with columns that will be used to answer our first question
values_to_analyse = ['Death_1_0','Attack_1_0']
keys = ['Country']
top_countries_attacks = raw_data_1.groupby(by=keys,as_index=False,)[values_to_analyse]\
                                    .sum()\
                                    .sort_values(by='Attack_1_0', ascending=False)\
                                    .head(10)
                                    #.style.hide_index()
top_countries_death = raw_data_1.groupby(by=keys,as_index=False,)[values_to_analyse]\
                                    .sum()\
                                    .sort_values(by='Death_1_0', ascending=False)\
                                    .head(10)
                                    #.style.hide_index()

In [91]:
#Answering question 1
answer_1 = top_countries_attacks
answer_1

,Country,Death_1_0,Attack_1_0
204,USA,186,2227
14,AUSTRALIA,283,1335
171,SOUTH AFRICA,105,577
145,PAPUA NEW GUINEA,56,132
127,NEW ZEALAND,24,128
23,BRAZIL,38,112
16,BAHAMAS,10,109
113,MEXICO,43,89
90,ITALY,14,71
61,FIJI,22,62


In [92]:
#Answering question 2
answer_2 = top_countries_death
answer_2

,Country,Death_1_0,Attack_1_0
14,AUSTRALIA,283,1335
204,USA,186,2227
171,SOUTH AFRICA,105,577
145,PAPUA NEW GUINEA,56,132
113,MEXICO,43,89
23,BRAZIL,38,112
149,PHILIPPINES,35,61
155,REUNION,29,59
127,NEW ZEALAND,24,128
45,CUBA,24,46


In [93]:
# Answering the second part from question 2
countries_death = list(answer_2.Country)
countries_attack = list (answer_1.Country)
diff_countries = [country for country in countries_death if country not in countries_attack]
diff_countries

['PHILIPPINES', 'REUNION', 'CUBA']

In [95]:
# Creating a dataframe with all the the countries in Top 10 attacks and deaths in order to answer question 3
values_to_analyse = ['Death_1_0','Attack_1_0']
keys = ['Country']
all_countries = pd.concat([answer_1,answer_2],ignore_index = True)
all_countries = all_countries.drop_duplicates()

In [96]:
# Answerting question 3
answer_3 = all_countries
answer_3['Death_Rate'] = answer_3.Death_1_0/answer_3.Attack_1_0
answer_3 = answer_3.sort_values(by='Death_Rate', ascending=False)
format_dict = {'Death_Rate': '{:.2%}'}
answer_3 = answer_3.style.format(format_dict).hide_index()
answer_3

Country,Death_1_0,Attack_1_0,Death_Rate
PHILIPPINES,35,61,57.38%
CUBA,24,46,52.17%
REUNION,29,59,49.15%
MEXICO,43,89,48.31%
PAPUA NEW GUINEA,56,132,42.42%
FIJI,22,62,35.48%
BRAZIL,38,112,33.93%
AUSTRALIA,283,1335,21.20%
ITALY,14,71,19.72%
NEW ZEALAND,24,128,18.75%


In [97]:
# Creating two lists: 1 for the top 10 countries with most attacks and another one for the top 10 in death historically. 
# This list will be used to filter our 'full dataframe' (with the rest of the columns), abling a vision 
# from Age, Year
list_countries = countries_attack + diff_countries
len(list_countries)

13

In [98]:
# Creating dataframe for further analysis.
df_final = raw_data_1[raw_data_1.Country.isin(list_countries)]
df_final

,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Case_Number.2,Original_Order,Adj_Year,Adj_Gender,Adj_Death,Adj_Age,Group_Year,Group_Age,Death_1_0,Attack_1_0
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,2018.06.25,6303.0,2018.0,F,N,57.0,"(2010, 2020]","(18.0, 60.0]",0,1
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson�McNeely,F,...,2018.06.18,6302.0,2018.0,F,N,11.0,"(2010, 2020]",NaN,0,1
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,2018.06.09,6301.0,2018.0,M,N,48.0,"(2010, 2020]","(18.0, 60.0]",0,1
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2018.06.08,6300.0,2018.0,M,N,NaN,"(2010, 2020]",NaN,0,1
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,2018.06.04,6299.0,2018.0,M,N,NaN,"(2010, 2020]",NaN,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6295,ND.0007,Before 1906,0.0,Unprovoked,AUSTRALIA,NaN,NaN,Fishing,fisherman,M,...,ND.0007,8.0,1906.0,M,Y,NaN,"(1900, 1910]",NaN,1,1
6296,ND.0006,Before 1906,0.0,Unprovoked,AUSTRALIA,New South Wales,,Swimming,Arab boy,M,...,ND.0006,7.0,1906.0,M,Y,NaN,"(1900, 1910]",NaN,1,1
6297,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,...,ND.0005,6.0,1903.0,M,Y,NaN,"(1900, 1910]",NaN,1,1
6298,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,...,ND.0004,5.0,1903.0,M,Y,NaN,"(1900, 1910]",NaN,1,1


In [99]:
len(df_final)/len(raw_data_1)

0.7961844197138315